## Setup for google colab

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
%cd /content/drive/My Drive/SA/Code/spoof_detection_deep_features/WaveletCNN

/content/drive/My Drive/SA/Code/spoof_detection_deep_features/WaveletCNN


## Run code from here

In [18]:
!pip install keras==2.1.5

  Using cached Keras-2.1.5-py2.py3-none-any.whl (334 kB)
  Attempting uninstall: keras
    Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  

In [2]:
import tensorflow as tf
sess = tf.Session()

from keras import backend as K
K.set_session(sess)

/home/rohita/rohit/spoof/work3/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rohita/rohit/spoof/work3/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rohita/rohit/spoof/work3/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/rohita/rohit/spoof/work3/lib/python3.5/site-packages

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import tensorflow as tf
import pandas as pd
import math
import numpy as np
from tensorflow.python.keras.layers import MaxPooling1D,MaxPooling2D,AveragePooling1D, Conv1D, LeakyReLU, BatchNormalization, Dense, Flatten, concatenate, Activation
from tensorflow.python.keras.layers import InputLayer, Input, Layer, Lambda, Dropout
from keras.utils import to_categorical
from tensorflow.python.keras.models import Model
from sklearn.preprocessing import StandardScaler
import tfwavelets
import keras
import tensorflow.contrib.slim as slim


In [4]:
def generator(data, labels, batch_size):
    """
    Yields the next training batch.
    Suppose `samples` is an array [[audio1,label1], [audio2,label2],...].
    """
    total_batches = int(data.shape[0])
    for i in range(total_batches):
        X_train,y_train = create_batches_rnd(data, labels, batch_size)
        yield X_train, y_train

In [5]:
def generator_val(data, labels, batch_size):
    """
    Yields the next training batch.
    Suppose `samples` is an array [[audio1,label1], [audio2,label2],...].
    """
    total_batches = int(data.shape[0])
    for i in range(total_batches):
        X_train,y_train = create_batches_rnd(data, labels, batch_size)
        yield X_train, y_train

In [6]:
def create_batches_rnd(data,labels,batch_size):
    wlen = 3200
    fact_amp = 0.2
    # Initialization of the minibatch (batch_size,[0=>x_t,1=>x_t+N,1=>random_samp])
    sig_batch=np.zeros([batch_size,wlen])
    lab_batch=[]
    signal_id_arr=np.random.randint(data.shape[0], size=batch_size)
    rand_amp_arr = np.random.uniform(1.0-fact_amp,1+fact_amp,batch_size)
    for i in range(batch_size): 
        signal = data[signal_id_arr[i]]
        # accesing to a random chunk
        signal_len=signal.shape[0]
        signal_beg=np.random.randint(signal_len-wlen-1) #randint(0, snt_len-2*wlen-1)
        signal_end=signal_beg+wlen
        sig_batch[i,:]=signal[signal_beg:signal_end]*rand_amp_arr[i]
        y=labels[signal_id_arr[i]]
        lab_batch.append(y)
    a, b = np.shape(sig_batch)
    sig_batch = sig_batch.reshape((a, b, 1))
    return sig_batch, np.array(lab_batch)

In [7]:
def res_conv_block(X,in_channels,out_channels,stage,block,dilation=1):

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    X_shortcut = X
    
    X = BatchNormalization(name=bn_name_base+'a')(X)
    X = Activation('relu')(X)
    X = Conv1D(in_channels, 3, padding='valid',use_bias = False, name= conv_name_base+'a')(X)
    X = BatchNormalization(name=bn_name_base+'b')(X)
    X = Activation('relu')(X)
    X = Conv1D(in_channels, 3, padding='valid',use_bias = False, name= conv_name_base+'b')(X)
    print(X.shape)
    paddings = tf.constant([[0, 0],   # the batch size dimension
                          [2, 2],   # top and bottom of image
                          [0, 0]])  # the channels dimension
    X = Lambda(lambda x: tf.pad(x, paddings, mode='CONSTANT',
                        constant_values=0.0))(X)
    X = concatenate([X , X_shortcut])
    X = BatchNormalization(name = bn_name_base+'c')(X)
    X = Activation('relu')(X)
    X = Conv1D(out_channels, 3, padding='valid',use_bias = False, dilation_rate = dilation, name = conv_name_base+'c')(X)

    return X



In [8]:
def down_block(input_layer, filters, kernel_size=3, activation="relu"):
  
        output = Conv1D(filters, kernel_size, padding="same", activation=activation)(input_layer)
        output = Conv1D(filters, kernel_size, padding="same", activation=activation)(output)
        output = Conv1D(filters, kernel_size, padding="same", activation=activation)(output)
        output = Conv1D(1,1,padding="same",activation=activation)(output)
#         print(output.shape,DWT_Pooling()(output).shape)
        return output, DWT_Pooling()(output)


def up_block(input_layer, residual_layer, filters, kernel_size=3,activation="relu"):
        output = Conv1D(1,1,padding="same",activation=activation)(input_layer)
        output = IWT_UpSampling()(output)
        output = concatenate([residual_layer,output])
        output = Conv1D(filters, kernel_size, padding="same", activation=activation)(output)
        output = Conv1D(filters, kernel_size, padding="same", activation=activation)(output)
        output = Conv1D(filters*2, kernel_size, padding="same", activation=activation)(output)
        return output

In [9]:
def dwt(x,db2):
    # x1_ = tf.placeholder(tf.float32, shape=(None,3200,3), name= 'x1')
    dwt = tfwavelets.nodes.dwt1d(x,db2,1)
    return dwt

    


def iwt(x,db2):
    
    idwt = tfwavelets.nodes.idwt1d(x,db2,1)
    return idwt
db2 = tfwavelets.dwtcoeffs.Wavelet(
    tfwavelets.dwtcoeffs.Filter(np.array([-0.12940952255092145,
                     0.22414386804185735,
                     0.836516303737469,
                     0.48296291314469025]), 3),
    tfwavelets.dwtcoeffs.Filter(np.array([-0.48296291314469025,
                     0.836516303737469,
                     -0.22414386804185735,
                     -0.12940952255092145]), 0),
    tfwavelets.dwtcoeffs.Filter(np.array([0.48296291314469025,
                     0.836516303737469,
                     0.22414386804185735,
                     -0.12940952255092145]), 0),
    tfwavelets.dwtcoeffs.Filter(np.array([-0.12940952255092145,
                     -0.22414386804185735,
                     0.836516303737469,
                     -0.48296291314469025]), 3)
)

class DWT_Pooling(Layer):
    """
    # Input shape :
        
            4D tensor of shape: (batch_size, signal, channels)
        
            
    # Output shape
        
            4D tensor of shape: (batch_size, signal/2, channels*4)
        
    """

    def __init__(self,**kwargs):
        super(DWT_Pooling, self).__init__(**kwargs)

    def build(self, input_shape):
        super(DWT_Pooling, self).build(input_shape) 

    def call(self, x):
        return dwt(x,db2)

    def compute_output_shape(self, input_shape):
        
        return (input_shape[0], input_shape[1]//2, input_shape[2]*4)


class IWT_UpSampling(Layer):
    """
    # Input shape :
        
            4D tensor of shape: (batch_size, signal, channels)
        
    # Output shape
        
            4D tensor of shape: (batch_size, singal*2, channels/4)
        
    """

    def __init__(self, **kwargs):
        super(IWT_UpSampling, self).__init__(**kwargs)

    def build(self, input_shape):
        super(IWT_UpSampling, self).build(input_shape) 

    def call(self, x):
        return iwt(x,db2)

    def compute_output_shape(self, input_shape):
        
        return (input_shape[0], input_shape[1]*2, input_shape[2]//4)

In [10]:
# def unetWavelet(input_size = (3200,1)):


input_size = None,3200,1    

# inputs = Input(shape = input_size)
inputs= tf.placeholder(tf.float32, shape=input_size, name= 'the_input')
down1, pool1 = down_block(inputs,8)
print(pool1.shape)
down2, pool2 = down_block(pool1,16)
down3, pool3 = down_block(pool2,32)
down4, pool4 = down_block(pool3,64)

down5 = Conv1D(filters=64, kernel_size= 3, padding="same", activation ="relu")(pool4)
down5 = Conv1D(filters=64, kernel_size= 3, padding="same", activation ="relu")(down5)
down5 = Conv1D(filters=32, kernel_size= 3, padding="same", activation ="relu")(down5)

# up = up_block(down5,down4,256)
# up = up_block(up,down3,128)
# up = up_block(up,down2,64)
# up = up_block(up,down1,32)

MWCNN_output = Conv1D(filters=input_size[1], kernel_size= 1, padding="same")(down5)

res_conv_1 = res_conv_block(MWCNN_output, 32, 16, 1, 'a', 4)
res_conv_2 = res_conv_block(res_conv_1, 16, 8, 2, 'a', 8)
res_conv_3 = res_conv_block(res_conv_2, 8, 4, 3, 'a', 16)
res_conv_4 = res_conv_block(res_conv_3, 4, 2, 4, 'a', 32)
res_conv_5 = res_conv_block(res_conv_4, 2, 1, 5, 'a', 64)

res_norm = BatchNormalization(name='res_norm')(res_conv_5)
res_relu = Activation('relu')(res_norm)





#level three decomposition starts 


pool_5_1 = AveragePooling1D(pool_size=3, padding='same', name='avg_pool_5_1')(res_relu)
flat_5_1 = Flatten(name='flat_5_1')(pool_5_1) 

fc_5 = Dense(2048, name='fc_5',kernel_initializer = tf.keras.initializers.glorot_uniform(seed=0))(flat_5_1)
norm_5 = BatchNormalization(name='norm_5')(fc_5)
relu_5 = Activation('relu', name='relu_5')(norm_5)
drop_5 = Dropout(0.5, name='drop_5')(relu_5)

fc_6 = Dense(2048, name='fc_6',kernel_initializer = tf.keras.initializers.glorot_uniform(seed=0))(drop_5)
norm_6 = BatchNormalization(name='norm_6')(fc_6)
relu_6 = Activation('relu', name='relu_6')(norm_6)
drop_6 = Dropout(0.5, name='drop_6')(relu_6)

output = Dense(2, activation=tf.nn.softmax)(drop_6)
# model = Model(input = inputs, output = output)
    
# return model

(?, 3200, 1)
(?, 3196, 32)
(?, 3188, 16)
(?, 3172, 8)
(?, 3140, 4)
(?, 3076, 2)


In [11]:
def model_summary():
    model_vars = tf.trainable_variables()
    slim.model_analyzer.analyze_vars(model_vars, print_info=True)

model_summary()

---------
Variables: name (type shape) [size]
---------
conv1d/kernel:0 (float32_ref 3x1x8) [24, bytes: 96]
conv1d/bias:0 (float32_ref 8) [8, bytes: 32]
conv1d_1/kernel:0 (float32_ref 3x8x8) [192, bytes: 768]
conv1d_1/bias:0 (float32_ref 8) [8, bytes: 32]
conv1d_2/kernel:0 (float32_ref 3x8x8) [192, bytes: 768]
conv1d_2/bias:0 (float32_ref 8) [8, bytes: 32]
conv1d_3/kernel:0 (float32_ref 1x8x1) [8, bytes: 32]
conv1d_3/bias:0 (float32_ref 1) [1, bytes: 4]
conv1d_4/kernel:0 (float32_ref 3x1x16) [48, bytes: 192]
conv1d_4/bias:0 (float32_ref 16) [16, bytes: 64]
conv1d_5/kernel:0 (float32_ref 3x16x16) [768, bytes: 3072]
conv1d_5/bias:0 (float32_ref 16) [16, bytes: 64]
conv1d_6/kernel:0 (float32_ref 3x16x16) [768, bytes: 3072]
conv1d_6/bias:0 (float32_ref 16) [16, bytes: 64]
conv1d_7/kernel:0 (float32_ref 1x16x1) [16, bytes: 64]
conv1d_7/bias:0 (float32_ref 1) [1, bytes: 4]
conv1d_8/kernel:0 (float32_ref 3x1x32) [96, bytes: 384]
conv1d_8/bias:0 (float32_ref 32) [32, bytes: 128]
conv1d_9/kerne

In [12]:
labels = tf.placeholder(tf.float32, shape=(None,2))

from keras.metrics import categorical_accuracy as accuracy


with tf.name_scope('Accuracy'):
    acc= tf.reduce_mean(accuracy(labels, output))

from keras.objectives import categorical_crossentropy
with tf.name_scope('Loss'):
    loss = tf.reduce_mean(categorical_crossentropy(labels, output))

In [13]:
X_train = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_train.npy")
y_train = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_train_labels.npy")
y_train = to_categorical(y_train)
# X_dev_train = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_train.npy")
# X_dev_val = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_val.npy")
# y_dev_train = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_train_labels.npy")
# y_dev_val = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_val_labels.npy")
# dev_wpt_levels_data_train = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_wpt_levels_data_train.npy")
# dev_wpt_levels_data_val = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_wpt_levels_data_val.npy")
# wpt_levels_data_train = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_wpt_levels_data_train.npy")
X_val = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_val.npy")
y_val = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_val_labels.npy")
y_val = to_categorical(y_val)
# wpt_levels_data_val = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_wpt_levels_data_val.npy")

In [ ]:
with tf.name_scope('RMSProp'):
    # Gradient Descent
    optimizer = tf.train.RMSPropOptimizer(0.001)
    train_step = optimizer.minimize(loss)
    # Op to calculate every variable gradient

tf.summary.scalar("loss", loss)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", acc)
# Create summaries to visualize weights
for var in tf.trainable_variables():
    tf.summary.histogram(var.name, var)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

# Initialize all variables
init_op = tf.global_variables_initializer()

batch_size = 32
logs_path = '/home/rohita/rohit/spoof/npy_data_asvspoof/MWCNN'

# Run training loop
with sess.as_default():
    
    sess.run(init_op)
    
    train_summary_writer = tf.summary.FileWriter(logs_path+'/Train',
                                            graph=tf.get_default_graph())
    val_summary_writer = tf.summary.FileWriter(logs_path+'/Val')
    gen = generator(X_train, y_train, batch_size)
    gen_val = generator_val(X_val, y_val, batch_size)
    total_batch_train = int(X_train.shape[0]/batch_size)
    total_batch_val = int(X_val.shape[0]/batch_size)
    for epoch in range(100):
        for i in range(total_batch_train):
            X_batch, y_batch = next(gen)
            feed_dict = {inputs: X_batch, labels: y_batch, tf.keras.backend.learning_phase(): 1}
            sess.run(train_step,feed_dict)
            loss_train, acc_train, summary = (sess.run([loss, acc, merged_summary_op],feed_dict))
            train_summary_writer.add_summary(summary, epoch * total_batch_train + i)
            print("Epoch: "+str(epoch)+"step: "+str(i)+"Training loss: ",loss_train," ","Training accuracy"," ",acc_train)
        
#             loss_val,acc_val = (sess.run([loss,acc_value],feed_dict={inputs: X_batch, labels: y_batch, tf.keras.backend.learning_phase(): 0}))
#             print("Epoch: "+str(i)+"step: "+str(step)+"loss: ",loss_val," ","accuracy"," ",acc_val)
            
                    
        for i in range(total_batch_val):
            X_batch, y_batch = next(gen_val)
            loss_val, acc_val, summary = (sess.run([loss, acc, merged_summary_op],feed_dict={inputs: X_batch, labels: y_batch, tf.keras.backend.learning_phase(): 0}))
            val_summary_writer.add_summary(summary, epoch * total_batch_val + i)
            print("val loss: ",loss_val," ","val accuracy"," ",acc_val)
            

INFO:tensorflow:Summary name conv1d/kernel:0 is illegal; using conv1d/kernel_0 instead.
INFO:tensorflow:Summary name conv1d/bias:0 is illegal; using conv1d/bias_0 instead.
INFO:tensorflow:Summary name conv1d_1/kernel:0 is illegal; using conv1d_1/kernel_0 instead.
INFO:tensorflow:Summary name conv1d_1/bias:0 is illegal; using conv1d_1/bias_0 instead.
INFO:tensorflow:Summary name conv1d_2/kernel:0 is illegal; using conv1d_2/kernel_0 instead.
INFO:tensorflow:Summary name conv1d_2/bias:0 is illegal; using conv1d_2/bias_0 instead.
INFO:tensorflow:Summary name conv1d_3/kernel:0 is illegal; using conv1d_3/kernel_0 instead.
INFO:tensorflow:Summary name conv1d_3/bias:0 is illegal; using conv1d_3/bias_0 instead.
INFO:tensorflow:Summary name conv1d_4/kernel:0 is illegal; using conv1d_4/kernel_0 instead.
INFO:tensorflow:Summary name conv1d_4/bias:0 is illegal; using conv1d_4/bias_0 instead.
INFO:tensorflow:Summary name conv1d_5/kernel:0 is illegal; using conv1d_5/kernel_0 instead.
INFO:tensorflow:

INFO:tensorflow:Summary name fc_5/kernel:0 is illegal; using fc_5/kernel_0 instead.
INFO:tensorflow:Summary name fc_5/bias:0 is illegal; using fc_5/bias_0 instead.
INFO:tensorflow:Summary name norm_5/gamma:0 is illegal; using norm_5/gamma_0 instead.
INFO:tensorflow:Summary name norm_5/beta:0 is illegal; using norm_5/beta_0 instead.
INFO:tensorflow:Summary name fc_6/kernel:0 is illegal; using fc_6/kernel_0 instead.
INFO:tensorflow:Summary name fc_6/bias:0 is illegal; using fc_6/bias_0 instead.
INFO:tensorflow:Summary name norm_6/gamma:0 is illegal; using norm_6/gamma_0 instead.
INFO:tensorflow:Summary name norm_6/beta:0 is illegal; using norm_6/beta_0 instead.
INFO:tensorflow:Summary name dense/kernel:0 is illegal; using dense/kernel_0 instead.
INFO:tensorflow:Summary name dense/bias:0 is illegal; using dense/bias_0 instead.
Epoch: 0step: 0Training loss:  0.6931478   Training accuracy   0.53125
Epoch: 0step: 1Training loss:  0.6932707   Training accuracy   0.5625
Epoch: 0step: 2Training

Epoch: 0step: 104Training loss:  0.68118954   Training accuracy   0.5625
Epoch: 0step: 105Training loss:  0.6837256   Training accuracy   0.59375
Epoch: 0step: 106Training loss:  0.69390696   Training accuracy   0.5
Epoch: 0step: 107Training loss:  0.7236183   Training accuracy   0.3125
Epoch: 0step: 108Training loss:  0.6968291   Training accuracy   0.5
Epoch: 0step: 109Training loss:  0.6962511   Training accuracy   0.53125
Epoch: 0step: 110Training loss:  0.70409536   Training accuracy   0.5
Epoch: 0step: 111Training loss:  0.6945741   Training accuracy   0.59375
val loss:  0.6931479   val accuracy   0.5
val loss:  0.69291997   val accuracy   0.59375
val loss:  0.6933758   val accuracy   0.40625
val loss:  0.6926921   val accuracy   0.6875
val loss:  0.69329983   val accuracy   0.4375
val loss:  0.69307196   val accuracy   0.53125
val loss:  0.69291997   val accuracy   0.59375
val loss:  0.69284403   val accuracy   0.625
val loss:  0.6931479   val accuracy   0.5
val loss:  0.6929959

In [ ]:
!nvidia-smi

In [ ]:
from keras.metrics import categorical_accuracy as accuracy

acc_value = accuracy(labels, preds)
with sess.as_default():
    print(acc_value.eval(feed_dict={img: mnist_data.test.images,
                                    labels: mnist_data.test.labels}))

## Tensorboard event log for Train and validation

In [24]:
count = 0
for e in tf.train.summary_iterator('/home/rohita/rohit/spoof/npy_data_asvspoof/MWCNN/Train/events.out.tfevents.1585208776.anands-MS-7A93'):
    for v in e.summary.value:
        if v.tag == 'loss':
            count = count + 1
            print("Step: "+str(count),v)
        elif v.tag == 'accuracy':
            print(v)


Step: 1 tag: "loss"
simple_value: 0.6930637359619141

tag: "accuracy"
simple_value: 0.625

Step: 2 tag: "loss"
simple_value: 0.6933633089065552

tag: "accuracy"
simple_value: 0.4375

Step: 3 tag: "loss"
simple_value: 0.6930726170539856

tag: "accuracy"
simple_value: 0.5625

Step: 4 tag: "loss"
simple_value: 0.692475438117981

tag: "accuracy"
simple_value: 0.5625

Step: 5 tag: "loss"
simple_value: 0.6924018859863281

tag: "accuracy"
simple_value: 0.5625

Step: 6 tag: "loss"
simple_value: 0.6929730176925659

tag: "accuracy"
simple_value: 0.5

Step: 7 tag: "loss"
simple_value: 0.6934524178504944

tag: "accuracy"
simple_value: 0.4375

Step: 8 tag: "loss"
simple_value: 0.6943036317825317

tag: "accuracy"
simple_value: 0.375

Step: 9 tag: "loss"
simple_value: 0.694058895111084

tag: "accuracy"
simple_value: 0.4375

Step: 10 tag: "loss"
simple_value: 0.6923717856407166

tag: "accuracy"
simple_value: 0.625

Step: 11 tag: "loss"
simple_value: 0.6941204071044922

tag: "accuracy"
simple_value: 0.

Step: 279 tag: "loss"
simple_value: 0.6700649261474609

tag: "accuracy"
simple_value: 0.625

Step: 280 tag: "loss"
simple_value: 0.7127885818481445

tag: "accuracy"
simple_value: 0.4375

Step: 281 tag: "loss"
simple_value: 0.7072441577911377

tag: "accuracy"
simple_value: 0.375

Step: 282 tag: "loss"
simple_value: 0.674103856086731

tag: "accuracy"
simple_value: 0.46875

Step: 283 tag: "loss"
simple_value: 0.7425273656845093

tag: "accuracy"
simple_value: 0.25

Step: 284 tag: "loss"
simple_value: 0.7289282083511353

tag: "accuracy"
simple_value: 0.34375

Step: 285 tag: "loss"
simple_value: 0.6987020969390869

tag: "accuracy"
simple_value: 0.53125

Step: 286 tag: "loss"
simple_value: 0.7104647159576416

tag: "accuracy"
simple_value: 0.4375

Step: 287 tag: "loss"
simple_value: 0.6841495037078857

tag: "accuracy"
simple_value: 0.53125

Step: 288 tag: "loss"
simple_value: 0.6717585325241089

tag: "accuracy"
simple_value: 0.65625

Step: 289 tag: "loss"
simple_value: 0.6775264739990234

tag:

Step: 541 tag: "loss"
simple_value: 0.6889506578445435

tag: "accuracy"
simple_value: 0.59375

Step: 542 tag: "loss"
simple_value: 0.6931208372116089

tag: "accuracy"
simple_value: 0.5

Step: 543 tag: "loss"
simple_value: 0.6921864151954651

tag: "accuracy"
simple_value: 0.5

Step: 544 tag: "loss"
simple_value: 0.6989479660987854

tag: "accuracy"
simple_value: 0.4375

Step: 545 tag: "loss"
simple_value: 0.6835825443267822

tag: "accuracy"
simple_value: 0.625

Step: 546 tag: "loss"
simple_value: 0.695578396320343

tag: "accuracy"
simple_value: 0.5

Step: 547 tag: "loss"
simple_value: 0.6939908266067505

tag: "accuracy"
simple_value: 0.5

Step: 548 tag: "loss"
simple_value: 0.6982918977737427

tag: "accuracy"
simple_value: 0.5

Step: 549 tag: "loss"
simple_value: 0.6977675557136536

tag: "accuracy"
simple_value: 0.46875

Step: 550 tag: "loss"
simple_value: 0.7169022560119629

tag: "accuracy"
simple_value: 0.3125

Step: 551 tag: "loss"
simple_value: 0.6808499097824097

tag: "accuracy"
sim

Step: 802 tag: "loss"
simple_value: 0.695174515247345

tag: "accuracy"
simple_value: 0.4375

Step: 803 tag: "loss"
simple_value: 0.6941351890563965

tag: "accuracy"
simple_value: 0.46875

Step: 804 tag: "loss"
simple_value: 0.6936205625534058

tag: "accuracy"
simple_value: 0.46875

Step: 805 tag: "loss"
simple_value: 0.6910869479179382

tag: "accuracy"
simple_value: 0.5

Step: 806 tag: "loss"
simple_value: 0.6916635632514954

tag: "accuracy"
simple_value: 0.59375

Step: 807 tag: "loss"
simple_value: 0.6931107044219971

tag: "accuracy"
simple_value: 0.53125

Step: 808 tag: "loss"
simple_value: 0.690658450126648

tag: "accuracy"
simple_value: 0.625

Step: 809 tag: "loss"
simple_value: 0.689195990562439

tag: "accuracy"
simple_value: 0.59375

Step: 810 tag: "loss"
simple_value: 0.6934745907783508

tag: "accuracy"
simple_value: 0.5

Step: 811 tag: "loss"
simple_value: 0.6778621673583984

tag: "accuracy"
simple_value: 0.6875

Step: 812 tag: "loss"
simple_value: 0.6910701990127563

tag: "acc

Step: 1066 tag: "loss"
simple_value: 0.6850884556770325

tag: "accuracy"
simple_value: 0.6875

Step: 1067 tag: "loss"
simple_value: 0.693848192691803

tag: "accuracy"
simple_value: 0.53125

Step: 1068 tag: "loss"
simple_value: 0.6950738430023193

tag: "accuracy"
simple_value: 0.5

Step: 1069 tag: "loss"
simple_value: 0.6951322555541992

tag: "accuracy"
simple_value: 0.40625

Step: 1070 tag: "loss"
simple_value: 0.690660834312439

tag: "accuracy"
simple_value: 0.53125

Step: 1071 tag: "loss"
simple_value: 0.6953800916671753

tag: "accuracy"
simple_value: 0.46875

Step: 1072 tag: "loss"
simple_value: 0.6934548616409302

tag: "accuracy"
simple_value: 0.53125

Step: 1073 tag: "loss"
simple_value: 0.6930564641952515

tag: "accuracy"
simple_value: 0.53125

Step: 1074 tag: "loss"
simple_value: 0.6917461156845093

tag: "accuracy"
simple_value: 0.5

Step: 1075 tag: "loss"
simple_value: 0.6919740438461304

tag: "accuracy"
simple_value: 0.5625

Step: 1076 tag: "loss"
simple_value: 0.6892822980880

...

Step: 2369 tag: "loss"
simple_value: 0.6918627619743347

tag: "accuracy"
simple_value: 0.5

Step: 2370 tag: "loss"
simple_value: 0.6962893605232239

tag: "accuracy"
simple_value: 0.40625

Step: 2371 tag: "loss"
simple_value: 0.6876461505889893

tag: "accuracy"
simple_value: 0.59375

Step: 2372 tag: "loss"
simple_value: 0.690876841545105

tag: "accuracy"
simple_value: 0.625

Step: 2373 tag: "loss"
simple_value: 0.6882169246673584

tag: "accuracy"
simple_value: 0.59375

Step: 2374 tag: "loss"
simple_value: 0.6957978010177612

tag: "accuracy"
simple_value: 0.4375

Step: 2375 tag: "loss"
simple_value: 0.689697265625

tag: "accuracy"
simple_value: 0.5

Step: 2376 tag: "loss"
simple_value: 0.6897730231285095

tag: "accuracy"
simple_value: 0.5

Step: 2377 tag: "loss"
simple_value: 0.6863729953765869

tag: "accuracy"
simple_value: 0.625

Step: 2378 tag: "loss"
simple_value: 0.6896221041679382

tag: "accuracy"
simple_value: 0.5

Step: 2379 tag: "loss"
simple_value: 0.6939065456390381

tag: "acc

Step: 2626 tag: "loss"
simple_value: 0.6869885921478271

tag: "accuracy"
simple_value: 0.71875

Step: 2627 tag: "loss"
simple_value: 0.6915004849433899

tag: "accuracy"
simple_value: 0.53125

Step: 2628 tag: "loss"
simple_value: 0.6967151165008545

tag: "accuracy"
simple_value: 0.4375

Step: 2629 tag: "loss"
simple_value: 0.6908268928527832

tag: "accuracy"
simple_value: 0.4375

Step: 2630 tag: "loss"
simple_value: 0.6927911043167114

tag: "accuracy"
simple_value: 0.53125

Step: 2631 tag: "loss"
simple_value: 0.6912516355514526

tag: "accuracy"
simple_value: 0.59375

Step: 2632 tag: "loss"
simple_value: 0.6951248049736023

tag: "accuracy"
simple_value: 0.5625

Step: 2633 tag: "loss"
simple_value: 0.696151077747345

tag: "accuracy"
simple_value: 0.53125

Step: 2634 tag: "loss"
simple_value: 0.6859039068222046

tag: "accuracy"
simple_value: 0.71875

Step: 2635 tag: "loss"
simple_value: 0.6901828646659851

tag: "accuracy"
simple_value: 0.5

Step: 2636 tag: "loss"
simple_value: 0.693417251

Step: 2886 tag: "loss"
simple_value: 0.6790311932563782

tag: "accuracy"
simple_value: 0.78125

Step: 2887 tag: "loss"
simple_value: 0.694093644618988

tag: "accuracy"
simple_value: 0.53125

Step: 2888 tag: "loss"
simple_value: 0.6789673566818237

tag: "accuracy"
simple_value: 0.625

Step: 2889 tag: "loss"
simple_value: 0.7040153741836548

tag: "accuracy"
simple_value: 0.40625

Step: 2890 tag: "loss"
simple_value: 0.6872276067733765

tag: "accuracy"
simple_value: 0.625

Step: 2891 tag: "loss"
simple_value: 0.7030924558639526

tag: "accuracy"
simple_value: 0.40625

Step: 2892 tag: "loss"
simple_value: 0.6889384984970093

tag: "accuracy"
simple_value: 0.5

Step: 2893 tag: "loss"
simple_value: 0.6850939989089966

tag: "accuracy"
simple_value: 0.59375

Step: 2894 tag: "loss"
simple_value: 0.6921172142028809

tag: "accuracy"
simple_value: 0.46875

Step: 2895 tag: "loss"
simple_value: 0.6977828741073608

tag: "accuracy"
simple_value: 0.46875

Step: 2896 tag: "loss"
simple_value: 0.6850408911

Step: 3147 tag: "loss"
simple_value: 0.6883022785186768

tag: "accuracy"
simple_value: 0.53125

Step: 3148 tag: "loss"
simple_value: 0.684086263179779

tag: "accuracy"
simple_value: 0.5625

Step: 3149 tag: "loss"
simple_value: 0.6907538771629333

tag: "accuracy"
simple_value: 0.5625

Step: 3150 tag: "loss"
simple_value: 0.7043483257293701

tag: "accuracy"
simple_value: 0.40625

Step: 3151 tag: "loss"
simple_value: 0.6951510906219482

tag: "accuracy"
simple_value: 0.46875

Step: 3152 tag: "loss"
simple_value: 0.6909372210502625

tag: "accuracy"
simple_value: 0.5

Step: 3153 tag: "loss"
simple_value: 0.6985896229743958

tag: "accuracy"
simple_value: 0.46875

Step: 3154 tag: "loss"
simple_value: 0.6943928003311157

tag: "accuracy"
simple_value: 0.5625

Step: 3155 tag: "loss"
simple_value: 0.6890807747840881

tag: "accuracy"
simple_value: 0.53125

Step: 3156 tag: "loss"
simple_value: 0.6886346936225891

tag: "accuracy"
simple_value: 0.5625

Step: 3157 tag: "loss"
simple_value: 0.6947628259

tag: "accuracy"
simple_value: 0.5

Step: 3406 tag: "loss"
simple_value: 0.6915963888168335

tag: "accuracy"
simple_value: 0.5

Step: 3407 tag: "loss"
simple_value: 0.6936421394348145

tag: "accuracy"
simple_value: 0.5

Step: 3408 tag: "loss"
simple_value: 0.690578281879425

tag: "accuracy"
simple_value: 0.625

Step: 3409 tag: "loss"
simple_value: 0.6937109231948853

tag: "accuracy"
simple_value: 0.5

Step: 3410 tag: "loss"
simple_value: 0.6925235390663147

tag: "accuracy"
simple_value: 0.53125

Step: 3411 tag: "loss"
simple_value: 0.6754048466682434

tag: "accuracy"
simple_value: 0.6875

Step: 3412 tag: "loss"
simple_value: 0.6916172504425049

tag: "accuracy"
simple_value: 0.46875

Step: 3413 tag: "loss"
simple_value: 0.6915818452835083

tag: "accuracy"
simple_value: 0.53125

Step: 3414 tag: "loss"
simple_value: 0.7018687725067139

tag: "accuracy"
simple_value: 0.4375

Step: 3415 tag: "loss"
simple_value: 0.6929801106452942

tag: "accuracy"
simple_value: 0.46875

Step: 3416 tag: "loss"

In [26]:
count = 0
for e in tf.train.summary_iterator('/home/rohita/rohit/spoof/npy_data_asvspoof/MWCNN/Val/events.out.tfevents.1585208859.anands-MS-7A93'):
    for v in e.summary.value:
        if v.tag == 'loss':
            count = count + 1
            print("Step: "+str(count),v)
        elif v.tag == 'accuracy':
            print(v)

Step: 1 tag: "loss"
simple_value: 0.6901168823242188

tag: "accuracy"
simple_value: 0.59375

Step: 2 tag: "loss"
simple_value: 0.6932904124259949

tag: "accuracy"
simple_value: 0.5

Step: 3 tag: "loss"
simple_value: 0.692232608795166

tag: "accuracy"
simple_value: 0.53125

Step: 4 tag: "loss"
simple_value: 0.6954060792922974

tag: "accuracy"
simple_value: 0.4375

Step: 5 tag: "loss"
simple_value: 0.688001275062561

tag: "accuracy"
simple_value: 0.65625

Step: 6 tag: "loss"
simple_value: 0.692232608795166

tag: "accuracy"
simple_value: 0.53125

Step: 7 tag: "loss"
simple_value: 0.692232608795166

tag: "accuracy"
simple_value: 0.53125

Step: 8 tag: "loss"
simple_value: 0.6890591382980347

tag: "accuracy"
simple_value: 0.625

Step: 9 tag: "loss"
simple_value: 0.6932904124259949

tag: "accuracy"
simple_value: 0.5

Step: 10 tag: "loss"
simple_value: 0.6954060792922974

tag: "accuracy"
simple_value: 0.4375

Step: 11 tag: "loss"
simple_value: 0.6985795497894287

tag: "accuracy"
simple_value: 

Step: 175 tag: "loss"
simple_value: 0.6940494775772095

tag: "accuracy"
simple_value: 0.46875

Step: 176 tag: "loss"
simple_value: 0.6932325959205627

tag: "accuracy"
simple_value: 0.5

Step: 177 tag: "loss"
simple_value: 0.6940495371818542

tag: "accuracy"
simple_value: 0.46875

Step: 178 tag: "loss"
simple_value: 0.694866418838501

tag: "accuracy"
simple_value: 0.4375

Step: 179 tag: "loss"
simple_value: 0.690781831741333

tag: "accuracy"
simple_value: 0.59375

Step: 180 tag: "loss"
simple_value: 0.6899648308753967

tag: "accuracy"
simple_value: 0.625

Step: 181 tag: "loss"
simple_value: 0.6940495371818542

tag: "accuracy"
simple_value: 0.46875

Step: 182 tag: "loss"
simple_value: 0.6940494775772095

tag: "accuracy"
simple_value: 0.46875

Step: 183 tag: "loss"
simple_value: 0.68914794921875

tag: "accuracy"
simple_value: 0.65625

Step: 184 tag: "loss"
simple_value: 0.698134183883667

tag: "accuracy"
simple_value: 0.3125

Step: 185 tag: "loss"
simple_value: 0.6973172426223755

tag: "a

Step: 362 tag: "loss"
simple_value: 0.6941164135932922

tag: "accuracy"
simple_value: 0.46875

Step: 363 tag: "loss"
simple_value: 0.6958601474761963

tag: "accuracy"
simple_value: 0.40625

Step: 364 tag: "loss"
simple_value: 0.6949882507324219

tag: "accuracy"
simple_value: 0.4375

Step: 365 tag: "loss"
simple_value: 0.6932445168495178

tag: "accuracy"
simple_value: 0.5

Step: 366 tag: "loss"
simple_value: 0.6949882507324219

tag: "accuracy"
simple_value: 0.4375

Step: 367 tag: "loss"
simple_value: 0.6906288862228394

tag: "accuracy"
simple_value: 0.59375

Step: 368 tag: "loss"
simple_value: 0.6958601474761963

tag: "accuracy"
simple_value: 0.40625

Step: 369 tag: "loss"
simple_value: 0.6932445168495178

tag: "accuracy"
simple_value: 0.5

Step: 370 tag: "loss"
simple_value: 0.6923726797103882

tag: "accuracy"
simple_value: 0.53125

Step: 371 tag: "loss"
simple_value: 0.6949882507324219

tag: "accuracy"
simple_value: 0.4375

Step: 372 tag: "loss"
simple_value: 0.6941163539886475

tag: 

Step: 547 tag: "loss"
simple_value: 0.692614734172821

tag: "accuracy"
simple_value: 0.53125

Step: 548 tag: "loss"
simple_value: 0.692614734172821

tag: "accuracy"
simple_value: 0.53125

Step: 549 tag: "loss"
simple_value: 0.692039966583252

tag: "accuracy"
simple_value: 0.5625

Step: 550 tag: "loss"
simple_value: 0.692614734172821

tag: "accuracy"
simple_value: 0.53125

Step: 551 tag: "loss"
simple_value: 0.6914651989936829

tag: "accuracy"
simple_value: 0.59375

Step: 552 tag: "loss"
simple_value: 0.6908904314041138

tag: "accuracy"
simple_value: 0.625

Step: 553 tag: "loss"
simple_value: 0.6943390369415283

tag: "accuracy"
simple_value: 0.4375

Step: 554 tag: "loss"
simple_value: 0.6954885721206665

tag: "accuracy"
simple_value: 0.375

Step: 555 tag: "loss"
simple_value: 0.6937642693519592

tag: "accuracy"
simple_value: 0.46875

Step: 556 tag: "loss"
simple_value: 0.6954885125160217

tag: "accuracy"
simple_value: 0.375

Step: 557 tag: "loss"
simple_value: 0.6926147937774658

tag: "

Step: 736 tag: "loss"
simple_value: 0.696262776851654

tag: "accuracy"
simple_value: 0.375

Step: 737 tag: "loss"
simple_value: 0.6955024003982544

tag: "accuracy"
simple_value: 0.40625

Step: 738 tag: "loss"
simple_value: 0.6932212114334106

tag: "accuracy"
simple_value: 0.5

Step: 739 tag: "loss"
simple_value: 0.6909400224685669

tag: "accuracy"
simple_value: 0.59375

Step: 740 tag: "loss"
simple_value: 0.6909400224685669

tag: "accuracy"
simple_value: 0.59375

Step: 741 tag: "loss"
simple_value: 0.69474196434021

tag: "accuracy"
simple_value: 0.4375

Step: 742 tag: "loss"
simple_value: 0.6901796460151672

tag: "accuracy"
simple_value: 0.625

Step: 743 tag: "loss"
simple_value: 0.6947420239448547

tag: "accuracy"
simple_value: 0.4375

Step: 744 tag: "loss"
simple_value: 0.6917004585266113

tag: "accuracy"
simple_value: 0.5625

Step: 745 tag: "loss"
simple_value: 0.69474196434021

tag: "accuracy"
simple_value: 0.4375

Step: 746 tag: "loss"
simple_value: 0.69474196434021

tag: "accurac

Step: 928 tag: "loss"
simple_value: 0.6924585103988647

tag: "accuracy"
simple_value: 0.53125

Step: 929 tag: "loss"
simple_value: 0.6916953325271606

tag: "accuracy"
simple_value: 0.5625

Step: 930 tag: "loss"
simple_value: 0.6955114603042603

tag: "accuracy"
simple_value: 0.40625

Step: 931 tag: "loss"
simple_value: 0.6932217478752136

tag: "accuracy"
simple_value: 0.5

Step: 932 tag: "loss"
simple_value: 0.6955114603042603

tag: "accuracy"
simple_value: 0.40625

Step: 933 tag: "loss"
simple_value: 0.690932035446167

tag: "accuracy"
simple_value: 0.59375

Step: 934 tag: "loss"
simple_value: 0.6909320950508118

tag: "accuracy"
simple_value: 0.59375

Step: 935 tag: "loss"
simple_value: 0.6932217478752136

tag: "accuracy"
simple_value: 0.5

Step: 936 tag: "loss"
simple_value: 0.6916953325271606

tag: "accuracy"
simple_value: 0.5625

Step: 937 tag: "loss"
simple_value: 0.6947481632232666

tag: "accuracy"
simple_value: 0.4375

Step: 938 tag: "loss"
simple_value: 0.6909320950508118

tag: "

Step: 1125 tag: "loss"
simple_value: 0.6914960145950317

tag: "accuracy"
simple_value: 0.5625

Step: 1126 tag: "loss"
simple_value: 0.6932450532913208

tag: "accuracy"
simple_value: 0.5

Step: 1127 tag: "loss"
simple_value: 0.6941195726394653

tag: "accuracy"
simple_value: 0.46875

Step: 1128 tag: "loss"
simple_value: 0.6906214952468872

tag: "accuracy"
simple_value: 0.59375

Step: 1129 tag: "loss"
simple_value: 0.6932450532913208

tag: "accuracy"
simple_value: 0.5

Step: 1130 tag: "loss"
simple_value: 0.6949940919876099

tag: "accuracy"
simple_value: 0.4375

Step: 1131 tag: "loss"
simple_value: 0.6897469162940979

tag: "accuracy"
simple_value: 0.625

Step: 1132 tag: "loss"
simple_value: 0.6941195726394653

tag: "accuracy"
simple_value: 0.46875

Step: 1133 tag: "loss"
simple_value: 0.6949940919876099

tag: "accuracy"
simple_value: 0.4375

Step: 1134 tag: "loss"
simple_value: 0.6932450532913208

tag: "accuracy"
simple_value: 0.5

Step: 1135 tag: "loss"
simple_value: 0.6914960145950317



Step: 1328 tag: "loss"
simple_value: 0.6964192390441895

tag: "accuracy"
simple_value: 0.46875

Step: 1329 tag: "loss"
simple_value: 0.6964192390441895

tag: "accuracy"
simple_value: 0.46875

Step: 1330 tag: "loss"
simple_value: 0.6964192390441895

tag: "accuracy"
simple_value: 0.46875

Step: 1331 tag: "loss"
simple_value: 0.6765543222427368

tag: "accuracy"
simple_value: 0.71875

Step: 1332 tag: "loss"
simple_value: 0.6964192390441895

tag: "accuracy"
simple_value: 0.46875

Step: 1333 tag: "loss"
simple_value: 0.7038686275482178

tag: "accuracy"
simple_value: 0.375

Step: 1334 tag: "loss"
simple_value: 0.6964192390441895

tag: "accuracy"
simple_value: 0.46875

Step: 1335 tag: "loss"
simple_value: 0.6939361691474915

tag: "accuracy"
simple_value: 0.5

Step: 1336 tag: "loss"
simple_value: 0.701385498046875

tag: "accuracy"
simple_value: 0.40625

Step: 1337 tag: "loss"
simple_value: 0.6914530396461487

tag: "accuracy"
simple_value: 0.53125

Step: 1338 tag: "loss"
simple_value: 0.70138549

Step: 1509 tag: "loss"
simple_value: 0.6931157112121582

tag: "accuracy"
simple_value: 0.53125

Step: 1510 tag: "loss"
simple_value: 0.6930210590362549

tag: "accuracy"
simple_value: 0.625

Step: 1511 tag: "loss"
simple_value: 0.6931473016738892

tag: "accuracy"
simple_value: 0.5

Step: 1512 tag: "loss"
simple_value: 0.6932419538497925

tag: "accuracy"
simple_value: 0.40625

Step: 1513 tag: "loss"
simple_value: 0.6933682560920715

tag: "accuracy"
simple_value: 0.28125

Step: 1514 tag: "loss"
simple_value: 0.6933998465538025

tag: "accuracy"
simple_value: 0.25

Step: 1515 tag: "loss"
simple_value: 0.6930525898933411

tag: "accuracy"
simple_value: 0.59375

Step: 1516 tag: "loss"
simple_value: 0.6931157112121582

tag: "accuracy"
simple_value: 0.53125

Step: 1517 tag: "loss"
simple_value: 0.6930526494979858

tag: "accuracy"
simple_value: 0.59375

Step: 1518 tag: "loss"
simple_value: 0.6932420134544373

tag: "accuracy"
simple_value: 0.40625

Step: 1519 tag: "loss"
simple_value: 0.6930209994